# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# lets implement my base model

## what do we need?
- first of all import neccesary libraries
- then write the main constants(.eg\ base-url, system-prompt)
- writing llm reponse funtion
    - we have to use history in our conversation for gradio setup
    

In [32]:
import json
from openai import OpenAI
import gradio as gr
from deep_translator import GoogleTranslator

# Configuration
BASE_URL = 'http://localhost:11434/v1'
MODEL = 'llama3.2'
API_KEY = 'ollama'

SYSTEM_PROMPT = """You are SafarBot, a bilingual flight assistant. 
Translate Persian<->English automatically when needed."""

class Translator:
    def __init__(self):
        self.translator = GoogleTranslator
    
    def translate(self, text: str, target_lang: str) -> str:
        try:
            source_lang = 'fa' if target_lang == 'en' else 'en'
            result = self.translator(source=source_lang, target=target_lang).translate(text)
            print(f"Translated ({source_lang}→{target_lang}): {text[:30]}... → {result[:30]}...")
            return result
        except Exception as e:
            print(f"⚠️ Translation failed: {e}")
            return text

def print_chat(messages):
    print("\n📜 Conversation:")
    for msg in messages:
        print(f"{msg['role'].upper()}: {msg['content'][:80]}{'...' if len(msg['content']) > 80 else ''}")
    print()

def chat(message: str, history: List[List[str]]) -> str:
    translator = Translator()
    client = OpenAI(base_url=BASE_URL, api_key=API_KEY)
    
    # Build message history
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    for user_msg, bot_msg in history:
        messages.extend([
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": bot_msg}
        ])
    
    # Handle current message
    is_persian = any(char in message for char in "پچجحخهعغفقثصضشسیبلاتنمکگوئدذرزطظژؤإأءً")
    
    if is_persian:
        messages.append({"role": "user", "content": translator.translate(message, 'en')})
    else:
        messages.append({"role": "user", "content": message})
    
    print_chat(messages)
    
    # Get LLM response
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    llm_response = response.choices[0].message.content
    print(f"🤖 LLM Response: {llm_response[:100]}...")
    
    return translator.translate(llm_response, 'fa') if is_persian else llm_response

# Launch the chat interface
gr.ChatInterface(
    fn=chat,
    title="✈️ SafarBot Flight Assistant",
    description="English/Persian flight booking assistant",
    examples=["Hello!", "سلام! پرواز به تهران دارید؟"],
    theme="soft"
).launch()

/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Translated (fa→en): سلام! پرواز به تهران دارید؟... → Hi! Do you have to fly to Tehr...

📜 Conversation:
SYSTEM: You are SafarBot, a bilingual flight assistant. 
Translate Persian<->English aut...
USER: Hi! Do you have to fly to Tehran?

🤖 LLM Response: Hello! I'm happy to help you with your travel plans.

There is no need to translate this simple ques...
Translated (en→fa): Hello! I'm happy to help you w... → سلام! خوشحالم که در برنامه های...
